# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow import random
random.set_seed(404)

In [2]:
import os
import numpy as np
import tensorflow as tf

# Constants

In [31]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE= 10000

IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = (IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS)

# Get the Data

In [4]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int) #read the text file and convert to numpy array

CPU times: total: 0 ns
Wall time: 37.6 ms


In [5]:
y_train_all.shape

(60000,)

In [6]:
y_test = np.loadtxt(Y_TEST_PATH,delimiter = ',',dtype=int)

In [7]:
y_test.shape

(10000,)

In [8]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)

CPU times: total: 3.77 s
Wall time: 6.87 s


In [9]:
%%time

x_test = np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

CPU times: total: 641 ms
Wall time: 1.01 s


In [13]:
x_test.shape

(10000, 784)

# Explore

In [14]:
x_train_all.shape

(60000, 784)

In [15]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [16]:
y_train_all.shape

(60000,)

In [17]:
x_test.shape

(10000, 784)

In [18]:
y_train_all[:5] # five first entry, our targer is sparse because has int for classes

array([5, 0, 4, 1, 9])

# Data Preprocessing

In [19]:
# Rescale

x_train_all, x_test = x_train_all/255.0, x_test/255.0

#### Convert target values to one-hot encoding

In [20]:
values = y_train_all[:5]
np.eye(10)[values]  # array element indexing  #the values in the square brackets acts as the index array.
# we're using this entire array as an index and we're pulling out several of the rows from the identity matrix.

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [21]:
np.eye(10) # why 10? because we have got 10 different labels in our dataset 0-9

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [22]:
np.eye(10)[9]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [23]:
values

array([5, 0, 4, 1, 9])

In [24]:
values[2]

4

In [25]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [26]:
y_train_all.shape

(60000, 10)

In [27]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

**Challenge:** Split the training dataset into a smaller training dataset and a validation dataset for the features and the labels. Create four arrays: ```x_val```, ```y_val```, ```x_train```, and ```y_train``` from ```x_train_all``` and ```y_train_all```. Use the validation size of 10,000. 

In [34]:
x_val =  x_train_all[:VALIDATION_SIZE] # first 10,000 value
y_val =  y_train_all[:VALIDATION_SIZE]

In [29]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [30]:
y_train.shape

(50000, 10)

# Setup Tensorflow Graph

In [40]:
# set features and labels as placeholders with variable batch size and data type tf.float32
X = tf.keras.Input(shape=(None, TOTAL_INPUTS), dtype=tf.float32)
Y = tf.keras.Input(shape=(None, NR_CLASSES), dtype=tf.float32)

### Neural Network Architecture

#### Hyperparameters

In [42]:
nr_epochs =5
learning_rate = 1e-4    # le-4 = 0.0001

# Nr neurons and layers
# the output layers is getting 10

n_hidden1 = 512
n_hidden2 = 64

In [43]:
initial_w = tf.random.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1], stddev=0.1, seed=42)
w = tf.Variable(initial_value=initial_w, name='W')
